## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [8]:
%load_ext autoreload
%autoreload

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbols=["AAPL", "AMD", "GOOGL", "PCT", "NVDA", "NKE", "CRWD", "TSLA", "MSFT", "JNJ", "TTEK"],
    interval="1d",
    start_date="2005-01-01",
    end_date="2024-01-01",
    membership="admin"
)
toolbox

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Toolbox(symbols=['AAPL', 'AMD', 'GOOGL', 'PCT', 'NVDA', 'NKE', 'CRWD', 'TSLA', 'MSFT', 'JNJ', 'TTEK'], interval=1d, start_date=2005-01-01, end_date=2024-01-01, provider=yfinance, membership=admin)

## Run Mandelbrot Channel Calculation

In [10]:
# import nest_asyncio
# nest_asyncio.apply()

result = toolbox.technical.mandelbrot_channel(historical=True, window="1m", _boundary_group_down=False, chart=False)
# result.to_polars()

INFO: MandelbrotChannelFetcher || START: fetch_data (sync)
INFO: MandelbrotChannelFetcher || END: fetch_data (sync) - Total time: 60.4014s


In [3]:
data = result.to_polars(equity_data=True)
data

date,open,high,low,close,volume,symbol
date,f64,f64,f64,f64,f64,str
2023-01-03,129.21547,129.830399,123.155395,124.048042,1.121175e8,"""AAPL"""
2023-01-03,66.0,66.879997,63.59,64.019997,4.68518e7,"""AMD"""
2023-01-03,107.5,108.459999,101.717003,103.279999,3.4475e6,"""CRWD"""
2023-01-03,89.487292,90.945625,88.418519,89.017838,2.81312e7,"""GOOGL"""
2023-01-03,168.322856,170.444092,168.179521,170.262543,6.3449e6,"""JNJ"""
…,…,…,…,…,…,…
2023-12-29,108.149466,109.142027,107.285935,107.762367,7.6609e6,"""NKE"""
2023-12-29,49.806573,49.990551,48.744709,49.51561,3.89293e8,"""NVDA"""
2023-12-29,4.15,4.24,4.02,4.05,1.7183e6,"""PCT"""


In [6]:
from humbldata.toolbox.technical.mandelbrot_channel.model import calc_mandelbrot_channel_historical_mp, calc_mandelbrot_channel_historical_concurrent


# result_mp = calc_mandelbrot_channel_historical_mp(
#     data=data,
#     window="1m",
#     rv_adjustment=True,
#     rv_method="std",
#     rs_method="RS",
#     rv_grouped_mean=False,
#     live_price=False
# )
result_cc = calc_mandelbrot_channel_historical_concurrent(
    data=data,
    window="1m",
    rv_adjustment=True,
    rv_method="std",
    rs_method="RS",
    rv_grouped_mean=False,
    live_price=False,
    use_processes=False,
    max_workers=16
)

In [7]:
# result_mp.collect()
result_cc.collect()

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2023-02-02,"""AAPL""",143.3908,149.587631,152.2181
2023-02-03,"""AAPL""",146.139,153.237579,154.5305
2023-02-06,"""AAPL""",145.8606,150.490189,156.4026
2023-02-07,"""AAPL""",147.7807,153.386353,156.5727
2023-02-08,"""AAPL""",147.7869,150.67865,158.0685
…,…,…,…,…
2023-12-22,"""TTEK""",158.1424,165.538162,168.7606
2023-12-26,"""TTEK""",158.6096,166.256165,169.5257
2023-12-27,"""TTEK""",158.083,167.921524,167.9215
